In [1]:
import sys
import pandas as pd
import numpy as np
from numba import jit
import json
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import xmltodict
import numpy.polynomial as p
from multiprocessing import Pool
import time
from datetime import datetime , date, timedelta

from fottech_lib.market_data.dmds import DMDSServices 
from fottech_lib import instrumentservice
from fottech_lib.market_data.repo import Repo

import project.market_data.repocurves as repoc
from project.market_data.repocurves import RepoCurves

In [2]:
def get_repo_schedules(ric,dates):
    print('############################## Index {} ##############################'.format(ric))
    dictionary = {}
    for business_date in dates:
        print(business_date)
        try:
            div_paths = 'RepoCurve/official/{}/PARIS/INTRADAY/equity/{}/sophis'.format(business_date,ric)
            ds = DMDSServices('prod', 'APAC')
            docs = ds.get_documents(div_paths)
            d_s = docs['documents']['document'][0].__values__.get('content')
            repo_schedule = xmltodict.parse(d_s)
            df = pd.DataFrame(repo_schedule['RepoCurve']['repo'])
            df['#text'] = df['#text'].astype(str)
            df['@term'] = df['@term'].astype(str)

            for i in range(df.shape[0]):
                f_date = datetime.strptime(business_date, "%Y%m%d").date()                
                l_date = datetime.strptime(df['@term'][i], "%Y-%m-%d").date()
                delta = l_date - f_date
                if (delta.days >= 0):
                    df['@term'][i] = delta.days
                else:
                    df = df.drop(i, axis = 0)
            df = df.reset_index(drop=True)
            df = df.get_values()
            col1 = df[:,0].tolist() 
            col2 = df[:,1].tolist() 
            col = [col1 , col2]
            dictionary[business_date]=col
        except:
            dictionary[business_date]=None
    return dictionary

In [9]:
def save_dict(dictionary, ric, past, future):
    file_path = 'data/processed/{}/Repo/repo_schedule_{}_from_{}_to_{}.json'.format(ric, ric,past,future)
    try:
        with open(file_path, 'w') as fp:
            json.dump(dictionary, fp)
        print('file saved')
    except:
        print('For some reasons, the file couldnt be saved')

In [91]:
index = 'UKX'
B_to_R = instrumentservice.InstrumentService('prod','APAC')
index_ric = B_to_R.transcode(index, target='reuter', partial_match=False)
ric = index_ric[1:]
past='20130101'
future = '20190726'
dates = repoc.business_dates(past=past)
dictionary = get_repo_schedules(ric,dates)
save_dict(dictionary, index, past, future)

############################## Index FTSE ##############################
20130101
20130102
20130103
20130104
20130107
20130108
20130109
20130110
20130111
20130114
20130115
20130116
20130117
20130118
20130121
20130122
20130123
20130124
20130125
20130128
20130129
20130130
20130131
20130201
20130204
20130205
20130206
20130207
20130208
20130211
20130212
20130213
20130214
20130215
20130218
20130219
20130220
20130221
20130222
20130225
20130226
20130227
20130228
20130301
20130304
20130305
20130306
20130307
20130308
20130311
20130312
20130313
20130314
20130315
20130318
20130319
20130320
20130321
20130322
20130325
20130326
20130327
20130328
20130329
20130401
20130402
20130403
20130404
20130405
20130408
20130409
20130410
20130411
20130412
20130415
20130416
20130417
20130418
20130419
20130422
20130423
20130424
20130425
20130426
20130429
20130430
20130501
20130502
20130503
20130506
20130507
20130508
20130509
20130510
20130513
20130514
20130515
20130516
20130517
20130520
20130521
20130522
20130523


20160617
20160620
20160621
20160622
20160623
20160624
20160627
20160628
20160629
20160630
20160701
20160704
20160705
20160706
20160707
20160708
20160711
20160712
20160713
20160714
20160715
20160718
20160719
20160720
20160721
20160722
20160725
20160726
20160727
20160728
20160729
20160801
20160802
20160803
20160804
20160805
20160808
20160809
20160810
20160811
20160812
20160815
20160816
20160817
20160818
20160819
20160822
20160823
20160824
20160825
20160826
20160829
20160830
20160831
20160901
20160902
20160905
20160906
20160907
20160908
20160909
20160912
20160913
20160914
20160915
20160916
20160919
20160920
20160921
20160922
20160923
20160926
20160927
20160928
20160929
20160930
20161003
20161004
20161005
20161006
20161007
20161010
20161011
20161012
20161013
20161014
20161017
20161018
20161019
20161020
20161021
20161024
20161025
20161026
20161027
20161028
20161031
20161101
20161102
20161103
20161104
20161107
20161108
20161109
20161110
20161111
20161114
20161115
20161116
20161117
20161118
2

In [92]:
print(len(dictionary.keys()))

1714
